In [70]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
data = pd.read_csv(r"D:\Data Science\NLP\classifier_bert\Amazon_mobile_reviews.csv")
data.head()

,Unnamed: 0,data/customer_id,data/helpful_votes,data/marketplace,data/product_category,data/product_id,data/product_parent,data/product_title,data/review_body,data/review_date,data/review_headline,data/review_id,data/star_rating,data/total_votes,data/verified_purchase,data/vine
0,0,b'20980074',0,b'US',b'Mobile_Electronics',b'B00D1847NE',b'274617424',b'Teenage Mutant Ninja Turtles Boombox CD Play...,b'Does not work',b'2015-01-09',b'One Star',b'R1OVS0D6SEXPW7',1,0,0,1
1,1,b'779273',0,b'US',b'Mobile_Electronics',b'B00KMO6DYG',b'397452138',b'4 Gauge Amp Kit Amplifier Install Wiring Com...,b'This is a great wiring kit i used it to set ...,b'2015-08-06',b'Great kit',b'R9VSD0ET8FERB',4,0,0,1
2,2,b'15410531',0,b'US',b'Mobile_Electronics',b'B000GWLL0K',b'948304826',b'Travel Wall Charger fits Creative Zen Vision...,b'It works great so much faster than USB charg...,b'2007-03-15',b'A/C Charger for Creative Zen Vision M',b'R3ISXCZHWLJLBH',5,0,0,1
3,3,b'27389005',0,b'US',b'Mobile_Electronics',b'B008L3JE6Y',b'466340015',b'High Grade Robust 360\xc2\xb0 Adjustable Car...,b'This product was purchased to hold a monitor...,b'2013-07-30',b'camera stand',b'R1TWVUDOFJSQAW',5,0,0,1
4,4,b'2663569',0,b'US',b'Mobile_Electronics',b'B00GHZS4SC',b'350592810',b'HDE Multifunctional Bluetooth FM Audio Car K...,"b""it works but it has really bad sound quality...",b'2014-12-31',b'bad sound quality',b'R2PEOEUR1LP0GH',3,0,0,1


In [20]:
data["sentiment"] = data["data/star_rating"].apply(lambda score : "positive" if score >=3 else "negative")
data["sentiment"] = data["sentiment"].map({'positive' : 1, "negative" : 0})
data["Review_text"] = data["data/review_body"].str.strip("b\'\"")


df = data[["Review_text", "sentiment"]]
df.head()

,Review_text,sentiment
0,Does not work,0
1,This is a great wiring kit i used it to set up...,1
2,It works great so much faster than USB charger...,1
3,This product was purchased to hold a monitor o...,1
4,it works but it has really bad sound quality. ...,1


In [34]:
seq_len = data["Review_text"].apply(lambda x : len(x.split()))


sns.set_style('darkgrid')
plt.figure(figsize = (200, 200))
sns.distplot(seq_len)

C:\Users\laksh\anaconda3\envs\classifier_bert\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


<AxesSubplot:xlabel='Review_text', ylabel='Density'>

In [42]:
SEQ_LEN = 100

In [23]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [43]:
tokens = tokenizer.encode_plus("Hello World", max_length=SEQ_LEN,truncation=True, padding="max_length", add_special_tokens=True,
                               return_tensors = 'tf', return_token_type_ids=False, return_attention_mask=True)

In [40]:
tokens.keys()

dict_keys(['input_ids', 'attention_mask'])

In [44]:
tokens

{'input_ids': <tf.Tensor: shape=(1, 100), dtype=int32, numpy=
array([[ 101, 8667, 1291,  102,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0]])>, 'attention_mask': <tf.Tensor: shape=(1, 100), dtype=int32, numpy=
array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [45]:
Xids = np.zeros((len(df), SEQ_LEN))
Xmask = np.zeros((len(df), SEQ_LEN))

In [46]:
Xids.shape

(104975, 100)

In [47]:
for i, sequence in enumerate(df["Review_text"]):
    tokens = tokenizer.encode_plus(sequence, max_length=SEQ_LEN,truncation=True, padding="max_length", add_special_tokens=True,
                               return_tensors = 'tf', return_token_type_ids=False, return_attention_mask=True)
    
    Xids[i,:], Xmask[i,:] = tokens['input_ids'],tokens['attention_mask']
   

In [48]:
Xids

array([[ 101., 7187., 1136., ...,    0.,    0.,    0.],
       [ 101., 1188., 1110., ...,    0.,    0.,    0.],
       [ 101., 1135., 1759., ...,    0.,    0.,    0.],
       ...,
       [ 101., 1109., 2267., ...,    0.,    0.,    0.],
       [ 101.,  146., 2802., ...,    0.,    0.,    0.],
       [ 101., 1135., 5521., ...,    0.,    0.,    0.]])

In [49]:
Xmask

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [50]:
df["sentiment"].unique()

array([0, 1], dtype=int64)

In [51]:
labels = np.zeros((len(df), 2))

labels[np.arange(len(df)), df["sentiment"].values] = 1

In [52]:
labels

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [54]:
with open('Xids.npy', 'wb') as f:
    np.save(f, Xids)
    
with open('Xmask.npy', 'wb') as f:
    np.save(f, Xmask)
    
with open('labels.npy', 'wb') as f:
    np.save(f, labels)

In [56]:
with open('Xids.npy', 'rb') as fp:
    xids = np.load(fp)
    
    
with open('Xmask.npy', 'rb') as fp:
    xmask = np.load(fp)
    
    
with open('labels.npy', 'rb') as fp:
    labels = np.load(fp)

In [57]:
xids

array([[ 101., 7187., 1136., ...,    0.,    0.,    0.],
       [ 101., 1188., 1110., ...,    0.,    0.,    0.],
       [ 101., 1135., 1759., ...,    0.,    0.,    0.],
       ...,
       [ 101., 1109., 2267., ...,    0.,    0.,    0.],
       [ 101.,  146., 2802., ...,    0.,    0.,    0.],
       [ 101., 1135., 5521., ...,    0.,    0.,    0.]])

In [61]:
dataset = tf.data.Dataset.from_tensor_slices((xids, xmask, labels))

In [62]:
for i in dataset.take(1):
    print(i)    

(<tf.Tensor: shape=(100,), dtype=float64, numpy=
array([ 101., 7187., 1136., 1250.,  102.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.])>, <tf.Tensor: shape=(100,), dtype=float64, numpy=
array([1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.

In [63]:
def map_func(input_ids, mask, labels):
    return{"input_ids" : input_ids, "attention_mask" : mask}, labels

In [64]:
dataset = dataset.map(map_func)

In [65]:
for i in dataset.take(1):
    print(i)

({'input_ids': <tf.Tensor: shape=(100,), dtype=float64, numpy=
array([ 101., 7187., 1136., 1250.,  102.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.])>, 'attention_mask': <tf.Tensor: shape=(100,), dtype=float64, numpy=
array([1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.

In [66]:
dataset = dataset.shuffle(10000).batch(32)

In [67]:
DS_LEN = len(list(dataset))

In [68]:
DS_LEN

3281

In [69]:
SPLIT = 0.9

train = dataset.take(round(DS_LEN * SPLIT))
valid = dataset.skip(round(DS_LEN * SPLIT))

In [72]:
bert = TFAutoModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [82]:
input_ids = tf.keras.layers.Input(shape = (SEQ_LEN,), name = 'input_ids', dtype = 'int32')
mask = tf.keras.layers.Input(shape = (SEQ_LEN,), name = 'attention_mask', dtype = 'int32')

embeddings = bert(input_ids, attention_mask = mask)[0]

X = tf.keras.layers.GlobalMaxPooling1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(0.1)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y = tf.keras.layers.Dense(2, activation = "softmax", name = "output")(X)


model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

In [83]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 100,                                         

In [84]:
model.layers[2].trainable = False

In [85]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 100,                                         

In [86]:
optimizer = tf.keras.optimizers.Adam(0.01)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')


model.compile(optimizer = optimizer, loss = loss, metrics = [acc])

In [ ]:
history = model.fit(train, validation_data=valid, epochs=2)

In [ ]:
tf.saved_model()